# Train CatBoost Model

In [1]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.catboost
from mlflow.models import infer_signature
import dotenv
import os

In [2]:
import catboost
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd

print('Python version:', os.sys.version)
print('Catboost version:', catboost.__version__)
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)

Python version: 3.9.22 (main, Jun 24 2025, 13:51:37) 
[Clang 17.0.0 (clang-1700.0.13.3)]
Catboost version: 1.1
Numpy version: 1.26.4
Pandas version: 1.5.3


## Configure

In [3]:
dotenv.load_dotenv()

True

In [4]:
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:8080")
MLFLOW_EXPERIMENT = "iris_classification"

In [5]:
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)

## Load Data

In [6]:
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train & Log Model

In [7]:
mlflow.set_experiment(MLFLOW_EXPERIMENT)

with mlflow.start_run():
    print("Starting MLflow run...")
    params = {
        "iterations": 100,
        "learning_rate": 0.1,
        "depth": 3,
    }
    mlflow.log_params(params)

    print("Training model...")
    model = CatBoostClassifier(**params, verbose=0) 
    model.fit(X_train, y_train)

    print("Evaluating model...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    print(f"Model accuracy: {accuracy:.4f}")

    print("Logging model to MLflow...")
    signature = infer_signature(X, y_pred)
    model_info = mlflow.catboost.log_model(model, "model", signature=signature)
    

/Users/ngin.chuan/repos/ray-serve-demo/.venv.py39/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
2025/06/24 17:40:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Starting MLflow run...
Training model...
Evaluating model...
Model accuracy: 1.0000
Logging model to MLflow...
🏃 View run legendary-roo-96 at: http://localhost:8080/#/experiments/433423518861823329/runs/535ec45f2dc0442bbd982d33381f1f1d
🧪 View experiment at: http://localhost:8080/#/experiments/433423518861823329


In [8]:
# Test predictions
X_test_sample = X_test.sample(5, random_state=42)
print('Test sample:')
display(X_test_sample)
print('Predictions:')
model.predict(X_test_sample)  

Test sample:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
143,6.8,3.2,5.9,2.3
56,6.3,3.3,4.7,1.6
128,6.4,2.8,5.6,2.1
69,5.6,2.5,3.9,1.1
68,6.2,2.2,4.5,1.5


Predictions:


/Users/ngin.chuan/repos/ray-serve-demo/.venv.py39/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


array([[2],
       [1],
       [2],
       [1],
       [1]])